# Add inchi formula to DE analysis results

### Load data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import outer_spacem as osm
import os

%matplotlib inline
%config InlineBackend.figure_formats = ['retina']

sns.set(
    rc={
        "figure.figsize":(5, 5),
        "legend.frameon": False
    },
    style="ticks",
    context="talk"
)

In [42]:
# Load adata
adata = sc.read("/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/Drug_W8/reprocessing/single_cell_analysis/spatiomolecular_adata.h5ad")

# MS_databases = {
#     "CoreMetabolome": "/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/molecules_databases/core_metabolome_v3.csv",
#     "SwissLipids": "/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/molecules_databases/swisslipids_2018-02-02-v2.tsv"
# }

MS_databases = {
    "CoreMetabolome": pd.read_csv(
    "/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/molecules_databases/core_metabolome_v3.csv",
    sep="\t"),
    "SwissLipids": pd.read_csv(
    "/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/molecules_databases/swisslipids_2018-02-02-v2.tsv",
    sep="\t")
}

In [43]:
added_column_name = "inchi"
var_df = adata.var.copy()

db_name = "CoreMetabolome"
db = MS_databases[db_name]


# for db_name in MS_databases:
#     db = MS_databases[db_name]




In [46]:
molecule_databases = [MS_databases[key] for key in MS_databases]



KeyboardInterrupt: 

In [53]:
new_df_data = {'annotation_id': [],
          'mol_index': [],
          'num_mols': [],
          'mol_id': []}

# Unroll lists in the adata.var dataframe:
for _, row in var_df.iterrows():
    ids = eval(row.moleculeIds)
    new_df_data["mol_id"] += ids
    new_df_data["mol_index"] += range(len(ids))
    new_df_data["annotation_id"] += [row.annotation_id for _ in range(len(ids))]
    new_df_data["num_mols"] += [len(ids) for _ in range(len(ids))]
    # idx_range = range(len(ids))


In [54]:
# Now merge with METASPACE molecule databases:
new_df = pd.DataFrame(new_df_data)
combined_molecules_db = pd.concat(molecule_databases)

In [58]:
combined_molecules_db.columns
# new_df

,Unnamed: 0,id,name,formula,inchi
0,1846.0,HMDB0003361,Pyrimidine,C4H4N2,InChI=1S/C4H4N2/c1-2-5-4-6-3-1/h1-4H
1,161.0,HMDB0000240,Sulfite,H2O3S,"InChI=1S/H2O3S/c1-4(2)3/h(H2,1,2,3)"
2,74263.0,HMDB0094691,Furanone A,C4H4O2,"InChI=1S/C4H4O2/c5-4-2-1-3-6-4/h1,3H,2H2"
3,917.0,HMDB0001184,Methyl propenyl ketone,C5H8O,"InChI=1S/C5H8O/c1-3-4-5(2)6/h3-4H,1-2H3/b4-3+"
4,7695.0,HMDB0012157,3-Methyl-2-butenal,C5H8O,"InChI=1S/C5H8O/c1-5(2)3-4-6/h3-4H,1-2H3"
...,...,...,...,...,...
320532,NaN,SLM:000509151,M(IP)2C(t20:0/22:1(13Z)),C60H115NO25P2,InChI=1S/C60H115NO25P2/c1-3-5-7-9-11-13-15-17-...
320533,NaN,SLM:000509152,M(IP)2C(t20:0/24:1(15Z)),C62H119NO25P2,InChI=1S/C62H119NO25P2/c1-3-5-7-9-11-13-15-17-...
320534,NaN,SLM:000509153,M(IP)2C(t20:0/26:1(17Z)),C64H123NO25P2,InChI=1S/C64H123NO25P2/c1-3-5-7-9-11-13-15-17-...
320535,NaN,SLM:000509154,M(IP)2C(t20:0/14:0(2OH)),C52H101NO26P2,InChI=1S/C52H101NO26P2/c1-3-5-7-9-11-13-15-16-...


In [60]:
merged_df = pd.merge(new_df, combined_molecules_db, how="left", left_on="mol_id", right_on="id")

In [69]:
merged_df

,annotation_id,mol_index,num_mols,mol_id,Unnamed: 0,id,name,formula,inchi
0,C4H9O7P+Na,0,1,HMDB0001321,1015.0,HMDB0001321,D-Erythrose 4-phosphate,C4H9O7P,"InChI=1S/C4H9O7P/c5-1-3(6)4(7)2-11-12(8,9)10/h..."
1,C5H12O7P2+H,0,4,HMDB0001120,871.0,HMDB0001120,Dimethylallylpyrophosphate,C5H12O7P2,"InChI=1S/C5H12O7P2/c1-5(2)3-4-11-14(9,10)12-13..."
2,C5H12O7P2+H,1,4,HMDB0001347,1035.0,HMDB0001347,Isopentenyl pyrophosphate,C5H12O7P2,"InChI=1S/C5H12O7P2/c1-5(2)3-4-11-14(9,10)12-13..."
3,C5H12O7P2+H,2,4,SLM:000000303,NaN,SLM:000000303,isopentenyl diphosphate,C5H12O7P2,"InChI=1S/C5H12O7P2/c1-5(2)3-4-11-14(9,10)12-13..."
4,C5H12O7P2+H,3,4,SLM:000000304,NaN,SLM:000000304,dimethylallyl diphosphate,C5H12O7P2,"InChI=1S/C5H12O7P2/c1-5(2)3-4-11-14(9,10)12-13..."
...,...,...,...,...,...,...,...,...,...
14082,C61H98O5+K,24,29,SLM:000192296,NaN,SLM:000192296,"TG(O-18:2(9Z,12Z)/22:6(4Z,7Z,10Z,13Z,16Z,19Z)/...",C61H98O5,InChI=1S/C61H98O5/c1-4-7-10-13-16-19-22-25-28-...
14083,C61H98O5+K,25,29,SLM:000192297,NaN,SLM:000192297,"TG(O-18:2(9Z,12Z)/22:6(4Z,7Z,10Z,13Z,16Z,19Z)/...",C61H98O5,InChI=1S/C61H98O5/c1-4-7-10-13-16-19-22-25-28-...
14084,C61H98O5+K,26,29,SLM:000192298,NaN,SLM:000192298,"TG(O-18:2(9Z,12Z)/18:3(6Z,9Z,12Z)/22:6(4Z,7Z,1...",C61H98O5,InChI=1S/C61H98O5/c1-4-7-10-13-16-19-22-25-28-...
14085,C61H98O5+K,27,29,SLM:000192299,NaN,SLM:000192299,"TG(O-18:2(9Z,12Z)/20:4(5Z,8Z,11Z,14Z)/20:5(5Z,...",C61H98O5,InChI=1S/C61H98O5/c1-4-7-10-13-16-19-22-25-28-...


In [89]:
inchi_lists = [None for _ in range(len(var_df))]
for _, row in merged_df.iterrows():
    idx = var_df.index.get_loc(row.annotation_id)
    inchi = inchi_lists[idx]
    if inchi is None:
        inchi = [None for _ in range(row.num_mols)]
    inchi[row.mol_index] = str(row.inchi)
    inchi_lists[idx] = inchi
    # var_df.loc[row.annotation_id, "inchi"] = inchi
    # var_df["inchi"][row.annotation_id] = inchi
    # break

# Finally, get data in the original format:
# df['x']['C'] = 10



In [90]:
var_df["inchi"] = inchi_lists

In [91]:
var_df

,annotation_id,formula,adduct,ionFormula,ion,mz,msm,fdr,databases,moleculeNames,moleculeIds,inchi
C4H9O7P+Na,C4H9O7P+Na,C4H9O7P,+Na,C4H9O7PNa,C4H9O7P+Na+,222.997771,0.034286,0.50,"[[""CoreMetabolome"", ""v3""]]","[""D-Erythrose 4-phosphate""]","[""HMDB0001321""]","[InChI=1S/C4H9O7P/c5-1-3(6)4(7)2-11-12(8,9)10/..."
C5H12O7P2+H,C5H12O7P2+H,C5H12O7P2,+H,C5H13O7P2,C5H12O7P2+H+,247.013063,0.448323,0.05,"[[""CoreMetabolome"", ""v3""], [""SwissLipids"", ""20...","[""Dimethylallylpyrophosphate"", ""Isopentenyl py...","[""HMDB0001120"", ""HMDB0001347"", ""SLM:000000303""...","[InChI=1S/C5H12O7P2/c1-5(2)3-4-11-14(9,10)12-1..."
C5H14NO4P+Na,C5H14NO4P+Na,C5H14NO4P,+Na,C5H14NO4PNa,C5H14NO4P+Na+,206.055226,0.448761,0.10,"[[""CoreMetabolome"", ""v3""]]","[""Phosphorylcholine""]","[""HMDB0001565""]","[InChI=1S/C5H14NO4P/c1-6(2,3)4-5-10-11(7,8)9/h..."
C6H13O7P+H,C6H13O7P+H,C6H13O7P,+H,C6H14O7P,C6H13O7P+H+,229.047126,0.001899,0.50,"[[""CoreMetabolome"", ""v3""], [""SwissLipids"", ""20...","[""(R)-5-phosphomevalonate"", ""Mevalonic acid-5P...","[""HMDB0001343"", ""SLM:000000301"", ""SLM:00002067...","[InChI=1S/C6H13O7P/c1-6(9,4-5(7)8)2-3-13-14(10..."
C7H13O8P+H,C7H13O8P+H,C7H13O8P,+H,C7H14O8P,C7H13O8P+H+,257.042041,0.057573,0.20,"[[""SwissLipids"", ""2018-02-02""]]","[""PA(2:0/2:0)""]","[""SLM:000025480""]",[InChI=1S/C7H13O8P/c1-5(8)13-3-7(15-6(2)9)4-14...
...,...,...,...,...,...,...,...,...,...,...,...,...
C56H99O7P+Na,C56H99O7P+Na,C56H99O7P,+Na,C56H99O7PNa,C56H99O7P+Na+,937.702024,0.004616,0.50,"[[""SwissLipids"", ""2018-02-02""]]","[""PA(P-17:0/36:6(18Z,21Z,24Z,27Z,30Z,33Z))""]","[""SLM:000046954""]",[InChI=1S/C56H99O7P/c1-4-5-6-7-8-9-10-11-12-13...
C57H97O7P+Na,C57H97O7P+Na,C57H97O7P,+Na,C57H97O7PNa,C57H97O7P+Na+,947.686374,0.011840,0.50,"[[""SwissLipids"", ""2018-02-02""]]","[""PA(P-18:2(9Z,12Z)/36:6(18Z,21Z,24Z,27Z,30Z,3...","[""SLM:000047034""]",[InChI=1S/C57H97O7P/c1-3-5-7-9-11-13-15-17-19-...
C57H99O7P+Na,C57H99O7P+Na,C57H99O7P,+Na,C57H99O7PNa,C57H99O7P+Na+,949.702024,0.181857,0.20,"[[""SwissLipids"", ""2018-02-02""]]","[""PA(O-18:2(9Z,12Z)/36:6(18Z,21Z,24Z,27Z,30Z,3...","[""SLM:000045474"", ""SLM:000046550"", ""SLM:000046...",[InChI=1S/C57H99O7P/c1-3-5-7-9-11-13-15-17-19-...
C57H101O7P+Na,C57H101O7P+Na,C57H101O7P,+Na,C57H101O7PNa,C57H101O7P+Na+,951.717674,0.091886,0.20,"[[""SwissLipids"", ""2018-02-02""]]","[""PA(O-18:1(11Z)/36:6(18Z,21Z,24Z,27Z,30Z,33Z)...","[""SLM:000044990"", ""SLM:000045074"", ""SLM:000045...",[InChI=1S/C57H101O7P/c1-3-5-7-9-11-13-15-17-19...


In [92]:
var_df.to_csv("/Users/alberto-mac/EMBL_ATeam/projects/gastrosome/Drug_W8/test_export.csv")